In [0]:

import pandas as pd

data= spark.read.table("silver_listings_tbl")


In [0]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression

assembler = VectorAssembler(inputCols=data.columns, outputCol="features")
data = assembler.transform(data)
train_data, test_data = data.randomSplit([2/3, 1/3], seed=0)

print("Training set count:", train_data.count())
print("Testing set count:", test_data.count())


Training set count: 12977
Testing set count: 6641


In [0]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import RegressionEvaluator


assembler = VectorAssembler(inputCols=data.columns, outputCol="features")

rf = RandomForestRegressor(featuresCol="features", labelCol="price")

paramGrid = ParamGridBuilder() \
    .addGrid(rf.numTrees, [50]) \
    .addGrid(rf.maxDepth, [5]) \
    .addGrid(rf.minInstancesPerNode, [ 2]) \
    .build()

evaluator = RegressionEvaluator(labelCol="price", predictionCol="prediction", metricName="rmse")

crossval = CrossValidator(estimator=rf,
                          estimatorParamMaps=paramGrid,
                          evaluator=evaluator,
                          numFolds=5)  # 5-fold cross-validation

cvModel = crossval.fit(train_data)

predictions = cvModel.transform(test_data)

rmse = evaluator.evaluate(predictions)
print("Root Mean Squared Error (RMSE) on test data =", rmse)




Root Mean Squared Error (RMSE) on test data = 215.13410654525177


In [0]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import RegressionEvaluator


assembler = VectorAssembler(inputCols=data.columns, outputCol="features")

rf = RandomForestRegressor(featuresCol="features", labelCol="price")

paramGrid = ParamGridBuilder() \
    .addGrid(rf.numTrees, [100,150]) \
    .addGrid(rf.maxDepth, [10,15]) \
    .addGrid(rf.minInstancesPerNode, [2,3]) \
    .build()

evaluator = RegressionEvaluator(labelCol="price", predictionCol="prediction", metricName="rmse")

crossval = CrossValidator(estimator=rf,
                          estimatorParamMaps=paramGrid,
                          evaluator=evaluator,
                          numFolds=5)  # 5-fold cross-validation

cvModel = crossval.fit(train_data)

predictions = cvModel.transform(test_data)

rmse = evaluator.evaluate(predictions)
print("Root Mean Squared Error (RMSE) on test data =", rmse)


